In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical

from Bio import SeqIO
from Bio.Data import IUPACData 
import csv
import numpy as np
import tensorflow as tf
import dask.dataframe as dd
import dask.array as da

#data_path = 'features_CENH3_DMR6_LUCA-CHLRE00002_orthologues.csv'
#data_path = 'features_oma-seqs-viridiplantae_test-7-8.csv'
#data_path = 'features_oma-seqs-viridiplantae_test-5-6-7-8.csv'
#data_path = 'features_oma-seqs-viridiplantae_test-4-5-6-7-8-9.csv'
data_path = 'features_oma-seqs-viridiplantae_test-4-5-6-7-8-9-10.csv'

Using TensorFlow backend.


In [2]:
def protein2integer(in_seq):
    
    ## define universe of possible input values
    all_protein_letters = list(IUPACData.extended_protein_letters)
    #print(all_protein_letters)
    ## define a mapping of chars to integers 
    ## i+1 beacuse we want to start from integer 1 instead of 0. 0 will be used for padding
    char_to_int = dict((c, i+1) for i, c in enumerate(all_protein_letters))
    int_to_char = dict((i+1, c) for i, c in enumerate(all_protein_letters))
    ## integer encode input data
    integer_encoded = [char_to_int[char] for char in in_seq.upper()]
    
    
    #return(integer_encoded,len(all_protein_letters))
    return(integer_encoded)
    

In [3]:
def make_dataset(in_file):
    with open(in_file, 'r') as f:
        reader = csv.reader(f, delimiter="\t")
        # get all the rows as a list
        d_set = list(reader)
        # transform data into numpy array
        d_set = np.array(d_set).astype(str)
        
    integer_encoded_proteins = np.array([protein2integer(seq) for seq in d_set[:,1]])
    
    G = d_set[:, 0]
    X = integer_encoded_proteins
    Y = d_set[:, 2].astype(int)
                         
    return(d_set,G,X,Y)

In [4]:
def make_dataset_dask(in_file):
    data = dd.read_csv(in_file,sep='\t', header=None)
    df = data.compute().reset_index(drop=True)
    integer_encoded_proteins = da.from_array([protein2integer(seq) for seq in df.values[:,1]],chunks=1000)
    G = df.values[:,0]
    X = integer_encoded_proteins.compute()
    Y = df.values[:,2].astype(int)
                     
    return(df,G,X,Y)

In [5]:
def make_train_test_set_idea1(G,X,Y):
    
    # here we keep 80% of random indexes in train set and the rest in test set
    
    indices = np.random.permutation(X.shape[0])
    train_size = int(indices.size*0.80)
    train_idx, test_idx = indices[:train_size], indices[train_size:]
    #print(len(train_idx),len(test_idx))
    
    X_train, X_test = X[train_idx,], X[test_idx,]
    #print(X_train.shape,X_test.shape)
    
    y_train, y_test = Y[train_idx,], Y[test_idx,]
    
    #print(X[train_idx[0],])
    #print(Y[train_idx[0],])

    #print(X_train[0,])
    #print(y_train[0,])
    
    return(X_train,y_train,X_test,y_test)

In [6]:
def make_train_test_set_idea2(G,X,Y):
    
    # here we try to keep one item from each cluster in test set
    # the rest goes to train set
    
    test_idx = []
    train_idx = []
    seen_cluster_id = []
    
    for i in range(0,Y.shape[0]):
            if Y[i] in seen_cluster_id :
                train_idx.append(i)
            else:
                test_idx.append(i)
                seen_cluster_id.append(Y[i])
                
    #print(len(train_idx),len(test_idx))
    
    X_train, X_test = X[train_idx,], X[test_idx,]
    #print(X_train.shape,X_test.shape)
    
    y_train, y_test = Y[train_idx,], Y[test_idx,]
    
    #print(X[train_idx[0],])
    #print(Y[train_idx[0],])

    #print(X_train[0,])
    #print(y_train[0,])
    
    return(X_train,y_train,X_test,y_test)

In [7]:
def model1(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    # https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
    # create the model
    embedding_vecor_length = 4
    model = Sequential()
    model.add(Embedding(num_letters, embedding_vecor_length, input_length=fixed_seq_length))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(75))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())
    
#''' 
#    model.fit(X_train_new, y_train, epochs=in_epochs, batch_size=in_batch_size) #epochs=3, batch_size=64)
#    ## Final evaluation of the model
#    scores = model.evaluate(X_test_new, y_test, verbose=0)
#    print("Accuracy: %.2f%%" % (scores[1]*100))
#'''


    # Convert labels to categorical one-hot encoding & fit the model
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)

#    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
#    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, verbose=1)

    # evaluate the model
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    loss, accuracy = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    print('Accuracy: %f' % (accuracy*100))
    
    return()


In [8]:
def model2(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    # Initializing the Sequential model from KERAS.
    model = Sequential()

    # Creating a 16 neuron hidden layer with Linear Rectified activation function.
    #model.add(Dense(16, input_dim=1, init='uniform', activation='relu'))
    model.add(Dense(16, input_dim=fixed_seq_length, kernel_initializer='uniform', activation='relu'))

    # Creating a 8 neuron hidden layer.
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))

    # Adding a output layer.
    model.add(Dense(n_classes, kernel_initializer='uniform', activation='softmax'))
    
    # Compiling the model
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

    print(model.summary())
   
    # Convert labels to categorical one-hot encoding & fit the model
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)

    # fit the model
#    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
#    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, verbose=1)

    # evaluate the model
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    loss, accuracy = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    print('Accuracy: %f' % (accuracy*100))
    
    return()

In [9]:
def one_hot_matrix(labels,C):
    
    C = tf.constant(C,name="C")
    one_hot_matrix = tf.one_hot(labels,C,axis=1)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    
    return one_hot

In [10]:
def model3(X_train_new, y_train,X_test_new, y_test, batch_size =100, hm_epochs =100): # CNN: Convolutional Neural Networks
    # Number of nodes in each NN hidden layer
    n_nodes_hl1 = 1500
    n_nodes_hl2 = 1500
    n_nodes_hl3 = 1500

    # Number of orthology clusters
    #n_classes = len(np.unique(np.concatenate((y_train,y_test),axis=0)))     #2 or 3 or ...
    
    #y_all = np.concatenate((y_train,y_test),axis=0)
    #y_min = np.amin(y_all)
    #n_classes = np.amax(y_all-y_min)+1
    
    
    train_y = one_hot_matrix(y_train,n_classes)
    test_y = one_hot_matrix(y_test,n_classes)

    # Batch size and Epoch size for training the NN
    #batch_size = 100   #100
    #hm_epochs = 100    #1000

    # Initializing X and Y
    x = tf.placeholder('float')
    y = tf.placeholder('float')

    # Initializing NN layers
    hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([len(X_train_new[0]), n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'f_fum':n_nodes_hl3,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}


    
    
    l1 = tf.add(tf.matmul(x,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weight']), hidden_3_layer['bias'])
    l3 = tf.nn.relu(l3)

    prediction = tf.matmul(l3,output_layer['weight']) + output_layer['bias']

        
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #try:
        #    epoch = int(open(tf_log,'r').read().split('\n')[-2])+1
        #    print('STARTING:',epoch)
        #except:
        #    epoch = 1
        epoch = 1

        while epoch <= hm_epochs:
            epoch_loss = 1
            
            i=0
            while i < len(X_train_new):
                start = i
                end = i+batch_size
                batch_x = np.array(X_train_new[start:end])
                batch_y = np.array(train_y[start:end])

                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,y: batch_y})
                epoch_loss += c
                i+=batch_size
                
            
            print('Epoch ',epoch,' out of ',hm_epochs,'- loss:',epoch_loss)
 
            
            #with open(tf_log,'a') as f:
            #    f.write(str(epoch)+'\n') 
            epoch +=1
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        #print("\nModel saved in path: %s " % my_model_save_path)
        print('\nAccuracy:',accuracy.eval({x:X_test_new, y:test_y}) * 100)
    return()

In [11]:
def model4(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    
    # Convert labels to categorical one-hot encoding  
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    X_train_new_one_hot_labels = np.array([to_categorical(x, num_classes=num_letters) for x in X_train_new])
    X_test_new_one_hot_labels = np.array([to_categorical(x, num_classes=num_letters) for x in X_test_new])

    # create the model    
    model = Sequential()
    model.add(LSTM(75,input_shape=X_train_new_one_hot_labels[0].shape,return_sequences=True))
    model.add(LSTM(75))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())  

    # fit the model
    model.fit(X_train_new_one_hot_labels, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size, verbose=1)

    # evaluate the model
    loss, accuracy = model.evaluate(X_test_new_one_hot_labels, y_test_one_hot_labels, verbose=0)
    print('Accuracy: %f' % (accuracy*100))
    
    return()


In [12]:
def model5(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    # LSTM
    
    # Convert labels to categorical one-hot encoding  
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)

    # create the model    
    model = Sequential()
    model.add(Embedding(num_letters, output_dim=fixed_seq_length))
    model.add(LSTM(75,return_sequences=True))
    model.add(LSTM(75))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())

    # fit the model
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)

    # evaluate the model
    loss, accuracy = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    print('Accuracy: %f' % (accuracy*100))
    
    return()


In [13]:
def model6(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    # GRU
    
    # Convert labels to categorical one-hot encoding  
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    
    # create the model    
    model = Sequential()
    model.add(Embedding(num_letters, output_dim=fixed_seq_length))
    model.add(GRU(128, return_sequences=False))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())

    # fit the model
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)

    # evaluate the model
    loss, accuracy = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    print('Accuracy: %f' % (accuracy*100))
    
    return()


In [14]:
def use_model(model_json_file="model.json",model_h5_file="model.h5"):
    # load json and create model
    json_file = open(model_json_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(model_h5_file)
    print("Loaded model from disk")
 
    # evaluate loaded model on test data
    loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    score = loaded_model.evaluate(X, Y, verbose=0)
    print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
    
    return()

In [15]:
def model7(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10,model_json_file="model.json",model_h5_file="model.h5"): # RNN: Recurrent Neural Networks
    # GRU
    
    # Convert labels to categorical one-hot encoding  
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    
    # create the model    
    model = Sequential()
    model.add(Embedding(num_letters, output_dim=fixed_seq_length))
    model.add(GRU(128, return_sequences=False))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())

    # fit the model
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)

    # evaluate the model
    #loss, accuracy = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    #print('Accuracy: %f' % (accuracy*100))
    scores = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[0], scores[0])) # Loss
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100)) # Accuracy
 
    # serialize model to JSON
    model_json = model.to_json()
    with open(model_json_file, "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(model_h5_file)
    print("Saved model to disk")
    
    return()


In [16]:
def test_1(d_set):
    print(d_set[:,0])
    print(d_set[0:2,1])
    print(d_set[:,2].astype(int))
    print(d_set.shape)
    print(d_set[:,1].shape)
    return()

In [17]:
def test_2(d_set):
    integer_encoded_proteins = np.array([protein2integer(seq) for seq in d_set[:,1]])
    print(len(integer_encoded_proteins))
    print(integer_encoded_proteins[0])
    #np.array(integer_encoded_proteins).shape
    print(integer_encoded_proteins.shape)
    #protein2integer(dataset[:,1])
    return()

In [18]:
def test_3(G,X,Y):
    print(G.shape)
    print(X.shape)
    print(Y.shape)

    print(G[0:3,])
    print(X[0:3,])
    print(Y[0:3,])
    
    print('Maximum review length: {}'.format(len(max(X, key=len))))
    print('Minimum review length: {}'.format(len(min(X, key=len))))
    
    return()

In [19]:
def test_4(X_train_new,X_train):
    print(X_train_new.shape)
    print(X_train_new[0,:])
    print(X_train.shape)
    print(X_train[0,])
    return()


In [20]:
dataset, G, X, Y = make_dataset(data_path)
X_train,y_train,X_test,y_test = make_train_test_set_idea2(G,X,Y)

#print("============ Test 1 =======================")
#test_1(dataset)
#print("============ Test 2 =======================")
#test_2(dataset)
#print("============ Test 3 =======================")
#test_3(G,X,Y)



In [21]:
num_letters = len(list(IUPACData.extended_protein_letters)) # = 26
#fixed_seq_length = len(max(X, key=len)) # maximum
#fixed_seq_length = (sum(len(X[i,]) for i in range(X.shape[0]))/X.shape[0])  # average
fixed_seq_length = 1000
n_classes = int(np.amax(np.concatenate((y_train,y_test),axis=0))+1)
# truncate and pad input sequences
X_train_new = sequence.pad_sequences(X_train, maxlen=fixed_seq_length, padding='post', truncating='post')
X_test_new = sequence.pad_sequences(X_test, maxlen=fixed_seq_length, padding='post', truncating='post')
  
#print("============ Test 4 =======================")
#test_4(X_train_new,X_train)


In [ ]:
model1(X_train_new, y_train, X_test_new, y_test,256,1000)

In [30]:
model2(X_train_new, y_train, X_test_new, y_test,256,1000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 16)                16016     
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_5 (Dense)              (None, 15999)             143991    
Total params: 160,143
Trainable params: 160,143
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
9719/9719 [==============================] - 5s 500us/step - loss: 9.1196 - acc: 3.0867e-04
Epoch 2/1000
9719/9719 [==============================] - 5s 486us/step - loss: 7.8655 - acc: 2.0578e-04
Epoch 3/1000
9719/9719 [==============================] - 5s 468us/step - loss: 7.4646 - acc: 2.0578e-04
Epoch 4/1000
9719/9719 [==============================] - 4s 462us/step - loss: 

9719/9719 [==============================] - 4s 453us/step - loss: 4.6026 - acc: 0.0975
Epoch 75/1000
9719/9719 [==============================] - 4s 445us/step - loss: 4.5788 - acc: 0.0942
Epoch 76/1000
9719/9719 [==============================] - 4s 444us/step - loss: 4.5781 - acc: 0.0964
Epoch 77/1000
9719/9719 [==============================] - 5s 481us/step - loss: 4.5625 - acc: 0.0941
Epoch 78/1000
9719/9719 [==============================] - 5s 480us/step - loss: 4.5416 - acc: 0.1008
Epoch 79/1000
9719/9719 [==============================] - 4s 461us/step - loss: 4.5404 - acc: 0.0963
Epoch 80/1000
9719/9719 [==============================] - 4s 446us/step - loss: 4.5148 - acc: 0.1033
Epoch 81/1000
9719/9719 [==============================] - 4s 447us/step - loss: 4.5201 - acc: 0.0985
Epoch 82/1000
9719/9719 [==============================] - 4s 450us/step - loss: 4.4929 - acc: 0.1047
Epoch 83/1000
9719/9719 [==============================] - 5s 483us/step - loss: 4.4929 - acc: 0

9719/9719 [==============================] - 4s 463us/step - loss: 3.7237 - acc: 0.2078
Epoch 233/1000
9719/9719 [==============================] - 4s 444us/step - loss: 3.7143 - acc: 0.2038
Epoch 234/1000
9719/9719 [==============================] - 4s 442us/step - loss: 3.7112 - acc: 0.2095
Epoch 235/1000
9719/9719 [==============================] - 5s 469us/step - loss: 3.7009 - acc: 0.2110
Epoch 236/1000
9719/9719 [==============================] - 5s 487us/step - loss: 3.7220 - acc: 0.2048
Epoch 237/1000
9719/9719 [==============================] - 4s 462us/step - loss: 3.7061 - acc: 0.2103
Epoch 238/1000
9719/9719 [==============================] - 4s 457us/step - loss: 3.6920 - acc: 0.2102
Epoch 239/1000
9719/9719 [==============================] - 4s 446us/step - loss: 3.7036 - acc: 0.2085
Epoch 240/1000
9719/9719 [==============================] - 4s 438us/step - loss: 3.6978 - acc: 0.2143
Epoch 241/1000
9719/9719 [==============================] - 5s 480us/step - loss: 3.6930

9719/9719 [==============================] - 4s 441us/step - loss: 3.4616 - acc: 0.2594
Epoch 391/1000
9719/9719 [==============================] - 4s 441us/step - loss: 3.4400 - acc: 0.2568
Epoch 392/1000
9719/9719 [==============================] - 4s 455us/step - loss: 3.4491 - acc: 0.2523
Epoch 393/1000
9719/9719 [==============================] - 5s 480us/step - loss: 3.4527 - acc: 0.2533
Epoch 394/1000
9719/9719 [==============================] - 5s 506us/step - loss: 3.4605 - acc: 0.2513
Epoch 395/1000
9719/9719 [==============================] - 5s 465us/step - loss: 3.4607 - acc: 0.2469
Epoch 396/1000
9719/9719 [==============================] - 4s 435us/step - loss: 3.4479 - acc: 0.2598
Epoch 397/1000
9719/9719 [==============================] - 4s 443us/step - loss: 3.4697 - acc: 0.2524
Epoch 398/1000
9719/9719 [==============================] - 5s 468us/step - loss: 3.4388 - acc: 0.2587
Epoch 399/1000
9719/9719 [==============================] - 5s 477us/step - loss: 3.4678

9719/9719 [==============================] - 4s 434us/step - loss: 3.3070 - acc: 0.2819
Epoch 549/1000
9719/9719 [==============================] - 4s 442us/step - loss: 3.2967 - acc: 0.2871
Epoch 550/1000
9719/9719 [==============================] - 5s 464us/step - loss: 3.3111 - acc: 0.2873
Epoch 551/1000
9719/9719 [==============================] - 5s 477us/step - loss: 3.3081 - acc: 0.2815
Epoch 552/1000
9719/9719 [==============================] - 5s 466us/step - loss: 3.3061 - acc: 0.2819
Epoch 553/1000
9719/9719 [==============================] - 4s 455us/step - loss: 3.2698 - acc: 0.2889
Epoch 554/1000
9719/9719 [==============================] - 4s 437us/step - loss: 3.3195 - acc: 0.2825
Epoch 555/1000
9719/9719 [==============================] - 4s 435us/step - loss: 3.2998 - acc: 0.2832
Epoch 556/1000
9719/9719 [==============================] - 5s 470us/step - loss: 3.2970 - acc: 0.2879
Epoch 557/1000
9719/9719 [==============================] - 5s 477us/step - loss: 3.2861

9719/9719 [==============================] - 4s 457us/step - loss: 3.2098 - acc: 0.3012
Epoch 707/1000
9719/9719 [==============================] - 4s 442us/step - loss: 3.2065 - acc: 0.3038
Epoch 708/1000
9719/9719 [==============================] - 4s 441us/step - loss: 3.1804 - acc: 0.3091
Epoch 709/1000
9719/9719 [==============================] - 5s 469us/step - loss: 3.2237 - acc: 0.2977
Epoch 710/1000
9719/9719 [==============================] - 5s 478us/step - loss: 3.2157 - acc: 0.3023
Epoch 711/1000
9719/9719 [==============================] - 5s 472us/step - loss: 3.2011 - acc: 0.3038
Epoch 712/1000
9719/9719 [==============================] - 4s 454us/step - loss: 3.2026 - acc: 0.3081
Epoch 713/1000
9719/9719 [==============================] - 4s 437us/step - loss: 3.1986 - acc: 0.3055
Epoch 714/1000
9719/9719 [==============================] - 4s 438us/step - loss: 3.1977 - acc: 0.3044
Epoch 715/1000
9719/9719 [==============================] - 5s 469us/step - loss: 3.2005

9719/9719 [==============================] - 4s 442us/step - loss: 3.1504 - acc: 0.3100
Epoch 865/1000
9719/9719 [==============================] - 4s 437us/step - loss: 3.1314 - acc: 0.3085
Epoch 866/1000
9719/9719 [==============================] - 5s 480us/step - loss: 3.1239 - acc: 0.3169
Epoch 867/1000
9719/9719 [==============================] - 5s 478us/step - loss: 3.1274 - acc: 0.3180
Epoch 868/1000
9719/9719 [==============================] - 4s 460us/step - loss: 3.1318 - acc: 0.3159
Epoch 869/1000
9719/9719 [==============================] - 4s 447us/step - loss: 3.1224 - acc: 0.3155
Epoch 870/1000
9719/9719 [==============================] - 4s 449us/step - loss: 3.1295 - acc: 0.3146
Epoch 871/1000
9719/9719 [==============================] - 4s 447us/step - loss: 3.1056 - acc: 0.3250
Epoch 872/1000
9719/9719 [==============================] - 5s 481us/step - loss: 3.1271 - acc: 0.3167
Epoch 873/1000
9719/9719 [==============================] - 5s 478us/step - loss: 3.1313

()

In [31]:
model3(X_train_new, y_train, X_test_new, y_test,256,100)

('Epoch ', 1, ' out of ', 100, '- loss:', 378949849.5)
('Epoch ', 2, ' out of ', 100, '- loss:', 147761707.0)
('Epoch ', 3, ' out of ', 100, '- loss:', 73142033.0)
('Epoch ', 4, ' out of ', 100, '- loss:', 25956969.90625)
('Epoch ', 5, ' out of ', 100, '- loss:', 4834577.47265625)
('Epoch ', 6, ' out of ', 100, '- loss:', 969778.787109375)
('Epoch ', 7, ' out of ', 100, '- loss:', 411937.79150390625)
('Epoch ', 8, ' out of ', 100, '- loss:', 233341.42724609375)
('Epoch ', 9, ' out of ', 100, '- loss:', 150442.95275878906)
('Epoch ', 10, ' out of ', 100, '- loss:', 105173.48291015625)
('Epoch ', 11, ' out of ', 100, '- loss:', 76847.17349243164)
('Epoch ', 12, ' out of ', 100, '- loss:', 57685.50369262695)
('Epoch ', 13, ' out of ', 100, '- loss:', 44328.76791572571)
('Epoch ', 14, ' out of ', 100, '- loss:', 34569.251779556274)
('Epoch ', 15, ' out of ', 100, '- loss:', 27380.43602848053)
('Epoch ', 16, ' out of ', 100, '- loss:', 21639.78512954712)
('Epoch ', 17, ' out of ', 100, '- l

()

In [32]:
model4(X_train_new, y_train, X_test_new, y_test,256,10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 1000, 75)          30600     
_________________________________________________________________
lstm_4 (LSTM)                (None, 75)                45300     
_________________________________________________________________
dense_6 (Dense)              (None, 15999)             1215924   
Total params: 1,291,824
Trainable params: 1,291,824
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
9719/9719 [==============================] - 116s 12ms/step - loss: 8.8293 - acc: 4.1156e-04
Epoch 2/10
9719/9719 [==============================] - 109s 11ms/step - loss: 7.8406 - acc: 2.0578e-04
Epoch 3/10
9719/9719 [==============================] - 109s 11ms/step - loss: 7.6903 - acc: 2.0578e-04
Epoch 4/10
9719/9719 [==============================] - 110s 11ms/step - loss: 

()

In [33]:
model5(X_train_new, y_train, X_test_new, y_test,256,50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 1000)        26000     
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 75)          322800    
_________________________________________________________________
lstm_6 (LSTM)                (None, 75)                45300     
_________________________________________________________________
dense_7 (Dense)              (None, 15999)             1215924   
Total params: 1,610,024
Trainable params: 1,610,024
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
9719/9719 [==============================] - 407s 42ms/step - loss: 8.8400 - acc: 0.0000e+00
Epoch 2/50
9719/9719 [==============================] - 407s 42ms/step - loss: 7.8420 - acc: 2.0578e-04
Epoch 3/50
9719/9719 [=============================

()

In [ ]:
model6(X_train_new, y_train, X_test_new, y_test,256,1000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 1000)        26000     
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               433536    
_________________________________________________________________
dense_8 (Dense)              (None, 15999)             2063871   
Total params: 2,523,407
Trainable params: 2,523,407
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
9719/9719 [==============================] - 390s 40ms/step - loss: 8.6985 - acc: 2.0578e-04
Epoch 2/1000
9719/9719 [==============================] - 389s 40ms/step - loss: 7.7785 - acc: 2.0578e-04
Epoch 3/1000
9719/9719 [==============================] - 389s 40ms/step - loss: 7.6847 - acc: 2.0578e-04
Epoch 4/1000
9719/9719 [==============================] - 389s 40ms/step 

9719/9719 [==============================] - 394s 41ms/step - loss: 2.6241 - acc: 0.4943
Epoch 153/1000
9719/9719 [==============================] - 394s 41ms/step - loss: 2.4909 - acc: 0.5291
Epoch 154/1000
9719/9719 [==============================] - 393s 40ms/step - loss: 2.5134 - acc: 0.5232
Epoch 155/1000
9719/9719 [==============================] - 394s 41ms/step - loss: 2.5713 - acc: 0.5080
Epoch 156/1000
9719/9719 [==============================] - 394s 41ms/step - loss: 2.4376 - acc: 0.5407
Epoch 157/1000
9719/9719 [==============================] - 394s 41ms/step - loss: 2.3997 - acc: 0.5495
Epoch 158/1000
9719/9719 [==============================] - 393s 40ms/step - loss: 2.3856 - acc: 0.5575
Epoch 159/1000
9719/9719 [==============================] - 394s 41ms/step - loss: 2.4097 - acc: 0.5475
Epoch 160/1000
9719/9719 [==============================] - 394s 41ms/step - loss: 2.4443 - acc: 0.5388
Epoch 161/1000
9719/9719 [==============================] - 393s 40ms/step - lo

In [ ]:
model7(X_train_new, y_train, X_test_new, y_test,256,10,"GRU_model_test.json",model_h5_file="GRU_model_test.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 1000)        26000     
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               433536    
_________________________________________________________________
dense_1 (Dense)              (None, 20372)             2627988   
Total params: 3,087,524
Trainable params: 3,087,524
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
12544/22579 [===============>..............] - ETA: 6:29 - loss: 9.4335 - acc: 1.5944e-04